# Free Response Answer Type

This notebook demonstrates how to generate open-ended questions with free-form text answers. Free response questions are perfect for open-ended analysis, explanation generation, and creative forecasting where you need detailed, nuanced answers.

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from lightningrod import LightningRod

api_key = os.getenv("LIGHTNINGROD_API_KEY")
if not api_key:
    raise ValueError("LIGHTNINGROD_API_KEY is not set")

client = LightningRod(api_key=api_key)

## Free Response Answer Type Overview

Free response questions return full text answers, not just single values or choices. They're ideal for:
- Open-ended analysis and explanations
- Creative forecasting
- Detailed scenario descriptions
- Questions that require nuanced, multi-sentence answers

Free response questions work with any data source: Google News, GDELT, or custom documents.

In [ ]:
from datetime import datetime, timedelta
from lightningrod import AnswerType, AnswerTypeEnum, NewsSeedGenerator

answer_type = AnswerType(answer_type=AnswerTypeEnum.FREE_RESPONSE)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query="technology industry trends",
)

In [ ]:
from lightningrod import QuestionGenerator, FilterCriteria

question_generator = QuestionGenerator(
    instructions=(
        "Generate open-ended forecasting questions that require detailed explanations. "
        "Questions should ask for analysis, predictions, or explanations that require multiple sentences to answer."
    ),
    examples=[
        "What will be the main theme of Apple's next product launch?",
        "Explain the potential impact of the new AI regulation on tech companies.",
        "What trends will dominate the technology industry in 2025?",
        "Describe how the recent merger will affect the competitive landscape.",
    ],
    bad_examples=[
        "Will Apple release a new product?",
        "What is AI?",
        "When will the product launch?",
    ],
    filter_=FilterCriteria(
        rubric="The question must require a detailed, multi-sentence answer",
        min_score=0.7
    ),
    answer_type=answer_type,
)

In [ ]:
from lightningrod import WebSearchLabeler, QuestionRenderer

labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

renderer = QuestionRenderer(answer_type=answer_type)

## Run the Pipeline

Generate free response questions from the news articles.

In [ ]:
from lightningrod import QuestionPipeline

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=20)

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [ ]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [ ]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
print(df.head())

## Use Cases for Free Response Questions

**Open-Ended Analysis:**
- "Explain the impact of X on Y"
- "What are the implications of Z?"

**Creative Forecasting:**
- "What will be the main theme of the next product launch?"
- "Describe how the industry will evolve"

**Detailed Predictions:**
- "What trends will dominate?"
- "How will the market change?"